In [1]:
import torch
import spacy
import pandas as pd
import re

In [2]:
device = torch.device('cpu')
if torch.cuda.is_available():
    device = torch.device('cuda')
torch.set_default_device(device)
print(f"Using device = {torch.get_default_device()}")

Using device = cpu


In [3]:
# file_path = 'MINDlarge_train/news.tsv'
# column_names = [
#     "News ID",
#     "Category",
#     "SubCategory",
#     "Title",
#     "Abstract",
#     "URL",
#     "Title Entities",
#     "Abstract Entities"
#
# ]
df = pd.read_csv("news_topic.csv")
df.head()


,News ID,Category,SubCategory,Title,Abstract,URL,Title Entities,Abstract Entities,news,tokens
0,N88753,lifestyle,lifestyleroyals,"The Brands Queen Elizabeth, Prince Charles, an...","Shop the notebooks, jackets, and more that the...",https://assets.msn.com/labs/mind/AAGH0ET.html,"[{""Label"": ""Prince Philip, Duke of Edinburgh"",...",[],"The Brands Queen Elizabeth, Prince Charles, an...","['the', 'brands', 'queen', 'elizabeth', ',', '..."
1,N45436,news,newsscienceandtechnology,Walmart Slashes Prices on Last-Generation iPads,Apple's new iPad releases bring big deals on l...,https://assets.msn.com/labs/mind/AABmf2I.html,"[{""Label"": ""IPad"", ""Type"": ""J"", ""WikidataId"": ...","[{""Label"": ""IPad"", ""Type"": ""J"", ""WikidataId"": ...",Walmart Slashes Prices on Last-Generation iPad...,"['walmart', 'slashes', 'prices', 'on', 'last',..."
2,N23144,health,weightloss,50 Worst Habits For Belly Fat,These seemingly harmless habits are holding yo...,https://assets.msn.com/labs/mind/AAB19MK.html,"[{""Label"": ""Adipose tissue"", ""Type"": ""C"", ""Wik...","[{""Label"": ""Adipose tissue"", ""Type"": ""C"", ""Wik...",50 Worst Habits For Belly Fat\nThese seemingly...,"['50', 'worst', 'habits', 'for', 'belly', 'fat..."
3,N86255,health,medical,Dispose of unwanted prescription drugs during ...,NaN,https://assets.msn.com/labs/mind/AAISxPN.html,"[{""Label"": ""Drug Enforcement Administration"", ...",[],Dispose of unwanted prescription drugs during ...,"['dispose', 'of', 'unwanted', 'prescription', ..."
4,N93187,news,newsworld,The Cost of Trump's Aid Freeze in the Trenches...,Lt. Ivan Molchanets peeked over a parapet of s...,https://assets.msn.com/labs/mind/AAJgNsz.html,[],"[{""Label"": ""Ukraine"", ""Type"": ""G"", ""WikidataId...",The Cost of Trump's Aid Freeze in the Trenches...,"['the', 'cost', 'of', 'trump', ""'s"", 'aid', 'f..."


In [ ]:
df.shape

In [4]:
df['Title'].isnull().sum()

np.int64(0)

In [6]:
print(pd.__version__)

2.2.3


In [5]:
df['Abstract'].fillna('', inplace=True)

C:\Users\Hp\AppData\Local\Temp\ipykernel_11316\3675929783.py:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['Abstract'].fillna('', inplace=True)


In [ ]:
def combine_title_abstract(row):
    title = re.sub(r'\s+', ' ', str(row['Title'])).strip()
    abstract = re.sub(r'\s+', ' ', str(row['Abstract'])).strip()
    return f"{title}\n{abstract}"

df['news'] = df.apply(combine_title_abstract, axis=1)

In [ ]:
df.head()

In [ ]:
print(df['news'][:200])

In [2]:
nlp = spacy.load("en_core_web_sm")

In [25]:
def filter(doc):
    tokenized_doc = nlp(doc)
    return [t for t in tokenized_doc if
          not t.is_space and \
          not t.is_stop and \
          not t.is_punct]

In [29]:
df['tokens'] = df['news'].apply(lambda text: [token.text.lower() for token in filter(text)])

In [24]:
print(df['tokens'].head())
print(df['tokens'].apply(type).value_counts())

0    ['brands', 'queen', 'elizabeth', 'prince', 'ch...
1    ['walmart', 'slashes', 'prices', 'generation',...
2    ['50', 'worst', 'habits', 'belly', 'fat', 'see...
3    ['dispose', 'unwanted', 'prescription', 'drugs...
4    ['cost', 'trump', 'aid', 'freeze', 'trenches',...
Name: tokens, dtype: object
tokens
<class 'str'>    101527
Name: count, dtype: int64


In [ ]:
df.head()

In [31]:
df.to_csv('news_topic.csv', index=False)

In [3]:
df = pd.read_csv('news_topic.csv')

In [32]:
df['tokens'][1]

['walmart',
 'slashes',
 'prices',
 'generation',
 'ipads',
 'apple',
 'new',
 'ipad',
 'releases',
 'bring',
 'big',
 'deals',
 'year',
 'models']

In [33]:
df['news'][1]

"Walmart Slashes Prices on Last-Generation iPads\nApple's new iPad releases bring big deals on last year's models."

In [34]:
for token in filter(df['news'][1]):
    print(f"Text: {token.text}, POS: {token.pos_}, Dep: {token.dep_}, Head: {token.head.text}")

Text: Walmart, POS: PROPN, Dep: nsubj, Head: Slashes
Text: Slashes, POS: VERB, Dep: ROOT, Head: Slashes
Text: Prices, POS: PROPN, Dep: dobj, Head: Slashes
Text: Generation, POS: PROPN, Dep: nmod, Head: Apple
Text: iPads, POS: PROPN, Dep: nummod, Head: Generation
Text: Apple, POS: PROPN, Dep: poss, Head: releases
Text: new, POS: ADJ, Dep: amod, Head: releases
Text: iPad, POS: PROPN, Dep: compound, Head: releases
Text: releases, POS: NOUN, Dep: pobj, Head: on
Text: bring, POS: VERB, Dep: advcl, Head: Slashes
Text: big, POS: ADJ, Dep: amod, Head: deals
Text: deals, POS: NOUN, Dep: dobj, Head: bring
Text: year, POS: NOUN, Dep: poss, Head: models
Text: models, POS: NOUN, Dep: pobj, Head: on


In [25]:
print(df["tokens"][3])

['dispose', 'unwanted', 'prescription', 'drugs', 'dea', 'day']


In [27]:
import ast
df['dict_tokens'] =  df["tokens"].apply(lambda x: ast.literal_eval(x) if isinstance(x, str) else x)

In [39]:
for tokens in df["tokens"]:
    print(tokens)
    print(type(tokens))
    if tokens:
        print(type(tokens[0]))
    break

['brands', 'queen', 'elizabeth', 'prince', 'charles', 'prince', 'philip', 'swear', 'shop', 'notebooks', 'jackets', 'royals', 'live']
<class 'str'>
<class 'str'>


In [41]:
print(df["tokens"].head())
print(type(df["dict_tokens"].iloc[0]))  # Should be <class 'list'>
print(type(df["dict_tokens"].iloc[0][0]))  # Should be <class 'str'>

0    ['brands', 'queen', 'elizabeth', 'prince', 'ch...
1    ['walmart', 'slashes', 'prices', 'generation',...
2    ['50', 'worst', 'habits', 'belly', 'fat', 'see...
3    ['dispose', 'unwanted', 'prescription', 'drugs...
4    ['cost', 'trump', 'aid', 'freeze', 'trenches',...
Name: tokens, dtype: object
<class 'list'>
<class 'str'>


In [30]:
print(df['dict_tokens'][4])

['cost', 'trump', 'aid', 'freeze', 'trenches', 'ukraine', 'war', 'lt', 'ivan', 'molchanets', 'peeked', 'parapet', 'sand', 'bags', 'line', 'war', 'ukraine', 'helmet', 'propped', 'trick', 'snipers', 'perforated', 'multiple', 'holes']


In [43]:
from gensim.corpora.dictionary import Dictionary

# Create a dictionary from tokens
dictionary = Dictionary(df["dict_tokens"])

# Filter extremes
dictionary.filter_extremes(no_below=1, no_above=0.5)

# Create a bag-of-words (BoW) corpus
corpus = [dictionary.doc2bow(doc) for doc in df["dict_tokens"]]

# Print results
print("Dictionary (token2id):", dictionary.token2id)
print("Corpus (first 5 entries):", corpus[:5])


Dictionary (token2id): {'brands': 0, 'charles': 1, 'elizabeth': 2, 'jackets': 3, 'live': 4, 'notebooks': 5, 'philip': 6, 'prince': 7, 'queen': 8, 'royals': 9, 'shop': 10, 'swear': 11, 'apple': 12, 'big': 13, 'bring': 14, 'deals': 15, 'generation': 16, 'ipad': 17, 'ipads': 18, 'models': 19, 'new': 20, 'prices': 21, 'releases': 22, 'slashes': 23, 'walmart': 24, 'year': 25, '50': 26, 'belly': 27, 'fat': 28, 'good': 29, 'habits': 30, 'harmless': 31, 'holding': 32, 'keeping': 33, 'seemingly': 34, 'shedding': 35, 'unwanted': 36, 'worst': 37, 'day': 38, 'dea': 39, 'dispose': 40, 'drugs': 41, 'prescription': 42, 'aid': 43, 'bags': 44, 'cost': 45, 'freeze': 46, 'helmet': 47, 'holes': 48, 'ivan': 49, 'line': 50, 'lt': 51, 'molchanets': 52, 'multiple': 53, 'parapet': 54, 'peeked': 55, 'perforated': 56, 'propped': 57, 'sand': 58, 'snipers': 59, 'trenches': 60, 'trick': 61, 'trump': 62, 'ukraine': 63, 'war': 64, 'affected': 65, 'destroyed': 66, 'fact': 67, 'felt': 68, 'fraud': 69, 'health': 70, 'he

In [45]:
from gensim.models.ldamodel import LdaModel

# Train the LDA model
lda_model = LdaModel(corpus=corpus, id2word=dictionary, num_topics=5, random_state=1)

# Print the topics with their keywords
topics = lda_model.print_topics(num_words=5)
for topic in topics:
    print(f"Topic {topic[0]}: {topic[1]}")


Topic 0: 0.013*"game" + 0.008*"week" + 0.007*"season" + 0.007*"sunday" + 0.006*"team"
Topic 1: 0.012*"state" + 0.009*"season" + 0.009*"football" + 0.007*"1" + 0.006*"team"
Topic 2: 0.017*"police" + 0.014*"said" + 0.012*"county" + 0.012*"man" + 0.010*"old"
Topic 3: 0.012*"new" + 0.009*"city" + 0.007*"fire" + 0.007*"$" + 0.006*"weather"
Topic 4: 0.015*"trump" + 0.011*"president" + 0.008*"impeachment" + 0.007*"new" + 0.006*"house"


In [47]:
from gensim import models, corpora
from gensim import similarities
from gensim.models.coherencemodel import CoherenceModel
from wordcloud import WordCloud

In [48]:
%%time
lda_model = models.ldamodel.LdaModel(corpus=corpus,
                                     id2word=dictionary,
                                     num_topics=3,
                                     passes=10,
                                     alpha='auto',
                                     eta='auto',
                                     random_state=1)

CPU times: total: 3min 34s
Wall time: 3min 50s


In [49]:
print(lda_model.alpha)
print(lda_model.eta)

[0.19266152 0.18535547 0.20173384]
[0.25725505 3.6963074  3.2323565  ... 0.16028681 0.16171841 0.16202334]


In [50]:
# View the words of the top topic from the previous article.
lda_model.show_topic(topics[0][0])

[('new', 0.010051583),
 ('$', 0.0052528414),
 ('year', 0.0048829587),
 ('veterans', 0.0047982396),
 ('day', 0.004662676),
 ('best', 0.0038486314),
 ('world', 0.003646921),
 ('like', 0.0036166825),
 ('time', 0.0035922478),
 ('years', 0.0035630127)]

In [51]:
# View the words of the second-most prevalent topic from the previous article.
lda_model.show_topic(topics[1][0])

[('game', 0.009276301),
 ('trump', 0.0079804),
 ('season', 0.0070818677),
 ('week', 0.006718392),
 ('state', 0.006479345),
 ('president', 0.0060250964),
 ('team', 0.0057532554),
 ('win', 0.005276849),
 ('football', 0.005108846),
 ('impeachment', 0.004333067)]

In [58]:
import pyLDAvis
import pyLDAvis.gensim_models as gensimvis
from gensim import models

In [59]:
# Visualize the topics
vis = gensimvis.prepare(lda_model, corpus, dictionary)

# Display the visualization
pyLDAvis.display(vis)